### Relative Affordability Calculaton

In [124]:
import pandas as pd
import numpy as np 
import geopy.distance
from geopy.distance import geodesic

pd.set_option('display.max_columns', None)

In [125]:
zips_data = pd.read_csv('../nicole/clean_data/final_zip_data.csv')

In [126]:
listings_data = pd.read_csv('../nicole/clean_data/redfin_cleaned_stage_1.csv')

/Users/nicole/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [127]:
smh_data = pd.read_csv('../nicole/clean_data/SMH_Projects.csv')
smh_data['MedianSalesPrice'] = (smh_data['MinBaseSalesPrice'] + smh_data['MaxBaseSalesPrice'])/2

smh_data.head()

,ProjectGroupID,AltProjectID,ProjectID,ElementName,Division,MetroAreaID,Latitude,Longitude,Address,city,State,state_fips,ZipCode,County,MetroAreaTitle,CountyName,county_fips,cbsa_code,cbsa_name,msa_name,ProductID,ProductDescription,Active,MinSquareFootage,MaxSquareFootage,MinBaseSalesPrice,MaxBaseSalesPrice,ProductTypeID,NumberBedrooms,NumberFullBaths,NumberHalfBaths,NumberGarageSpaces,MedianSalesPrice
0,MST,721,Main Street Townes at Lilburn TH,Main Street Townes at Lilburn TH (721),Atlanta,1007,33.891702,-84.140783,346 Main St NW,Lilburn,GA,13.0,30047.0,29.0,Atlanta,Gwinnett,135.0,12060.0,Atlanta-Sandy Springs-Alpharetta GA,Atlanta GA MSA,001562,Dupree,Y,2113,2191,385900.0,393500.0,NaN,NaN,NaN,NaN,NaN,389700.0
1,LD,LD,West Broad Landing 2o2,West Broad Landing 2o2 (LD),Richmond,1006,37.616200,-77.521000,"2636 Lassen Walk, Unit A",Henrico,VA,51.0,23294.0,20.0,Richmond,Henrico,87.0,40060.0,Richmond VA,Richmond-Petersburg VA MSA,279,Paxton,Y,2452,2855,354990.0,355790.0,1.0,3.0,2.0,1.0,1.0,355390.0
2,SCK,SCK3,Sage Creek SFD,Sage Creek SFD (SCK3),Aiken/Augusta,1013,33.593076,-81.824541,1182 Sapphire Drive,Graniteville,SC,45.0,29829.0,37.0,Aiken/North Augusta,Aiken,3.0,12260.0,Augusta-Richmond County GA-SC,Augusta-Aiken GA-SC MSA,001420,Barlow,Y,2273,2306,236900.0,244900.0,4.0,3.0,2.0,1.0,2.0,240900.0
3,VST,VST3,Vancouver Station at Trolley Run SFD,Vancouver Station at Trolley Run SFD (VST3),Aiken/Augusta,1013,33.582816,-81.776370,507 Burnaby Court,Aiken,SC,45.0,29801.0,37.0,Aiken/North Augusta,Aiken,3.0,12260.0,Augusta-Richmond County GA-SC,Augusta-Aiken GA-SC MSA,001423,Santee,Y,1295,1593,210900.0,213900.0,4.0,3.0,2.0,0.0,2.0,212400.0
4,VST,VST3,Vancouver Station at Trolley Run SFD,Vancouver Station at Trolley Run SFD (VST3),Aiken/Augusta,1013,33.582816,-81.776370,507 Burnaby Court,Aiken,SC,45.0,29801.0,37.0,Aiken/North Augusta,Aiken,3.0,12260.0,Augusta-Richmond County GA-SC,Augusta-Aiken GA-SC MSA,001421,Summerton,Y,2386,2386,244900.0,250900.0,4.0,3.0,2.0,1.0,0.0,247900.0


In [135]:
#dict of lat/long for each SMH property
smh_geo = smh_data[['MetroAreaID', 'Latitude', 'Longitude', 'MedianSalesPrice']]
#smh_geo_dict = smh_geo.set_index('MetroAreaID').T.to_dict('list')
smh_geo.head()
#smh_geo_dict

,MetroAreaID,Latitude,Longitude,MedianSalesPrice
0,1007,33.891702,-84.140783,389700.0
1,1006,37.616200,-77.521000,355390.0
2,1013,33.593076,-81.824541,240900.0
3,1013,33.582816,-81.776370,212400.0
4,1013,33.582816,-81.776370,247900.0


In [129]:
#dict of centroid lat/long for each SMH community/metro area ID
centroids = pd.read_csv('../project/centroid_dummy.csv')

#centroids_dict = centroids.set_index('MetroAreaID').T.to_dict('list')


centroids
#centroids_dict

,MetroAreaID,Latitude,Longitude
0,1001.0,52.229676,-21.012229
1,1006.0,37.541290,-77.434769
2,1007.0,33.748783,-84.388168
3,1004.0,35.884766,-78.625053
4,1000.0,38.805100,-77.047000
5,1003.0,38.033554,-78.507980
6,1011.0,35.227085,-80.843124
7,1016.0,28.538336,-81.379234
8,1014.0,39.739071,-75.539787
9,NaN,34.895900,-82.217200


In [141]:
#merge centroid with smh addresses
smh_centroid = smh_geo.merge(centroids, on='MetroAreaID', how="left")
smh_centroid.columns = ["MetroAreaID", "Latitude" ,"Longitude",'MedianSalesPrice', "centroid_lat", "centroid_long"]

smh_centroid.dropna(inplace = True)
smh_centroid.isna().sum()

smh_centroid

,MetroAreaID,Latitude,Longitude,MedianSalesPrice,centroid_lat,centroid_long
0,1007,33.891702,-84.140783,389700.0,33.748783,-84.388168
1,1006,37.616200,-77.521000,355390.0,37.541290,-77.434769
2,1013,33.593076,-81.824541,240900.0,33.507600,-81.860400
3,1013,33.582816,-81.776370,212400.0,33.507600,-81.860400
4,1013,33.582816,-81.776370,247900.0,33.507600,-81.860400
...,...,...,...,...,...,...
445,1001,38.300400,-76.549900,327990.0,39.045753,-76.641273
446,1004,35.794500,-78.940100,483990.0,35.884766,-78.625053
448,1000,38.907833,-77.496722,866490.0,38.805100,-77.047000
449,1000,38.797253,-77.652348,940490.0,38.805100,-77.047000


In [142]:
smh_centroid['lat_long'] = list(zip(smh_centroid.Latitude,smh_centroid.Longitude))
smh_centroid['centroid'] = list(zip(smh_centroid.centroid_lat,smh_centroid.centroid_long))
smh_centroid = smh_centroid.drop(["Latitude", "Longitude", "centroid_lat", "centroid_long"], axis=1)
smh_centroid['distance'] = smh_centroid.apply(lambda x: geopy.distance.geodesic(x.lat_long, x.centroid), axis = 1)

#df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)

#### Distance to Centroid & Median Sale Prices

In [143]:
#list of every SMH property and distance to centroid
smh_centroid.head(15)

,MetroAreaID,MedianSalesPrice,lat_long,centroid,distance
0,1007,389700.0,"(33.891702, -84.140783)","(33.748783, -84.388168)",27.85372953251022 km
1,1006,355390.0,"(37.6162, -77.521)","(37.54129, -77.434769)",11.275803895831249 km
2,1013,240900.0,"(33.593076, -81.824541)","(33.5076, -81.8604)",10.04839088810054 km
3,1013,212400.0,"(33.582816, -81.77637)","(33.5076, -81.8604)",11.423813153281548 km
4,1013,247900.0,"(33.582816, -81.77637)","(33.5076, -81.8604)",11.423813153281548 km
5,1008,289900.0,"(32.8984, -79.9979)","(32.776566, -79.930923)",14.895773216251829 km
6,1000,884990.0,"(38.7972531, -77.6523484)","(38.8051, -77.047)",52.59271546370725 km
7,1013,246900.0,"(33.5237447, -81.8765853)","(33.5076, -81.8604)",2.3382963395454497 km
8,1010,241900.0,"(33.917912, -81.328175)","(34.0007, -81.0348)",28.62901156133804 km
9,1000,640990.0,"(38.4305, -77.4289)","(38.8051, -77.047)",53.2485419990901 km


## Inputs

#### Average Resale Price:
- Average resale price of the competitor houses within the radius of selected distance  from the SMH community. (Previous Year Closing price) 

#### Average Stanley Martin Base Price: 
- Average base price of all SMH homes in the selected SMH Community. 

In [39]:
#list of average sale price of each SMH Community 
smh = smh_data.groupby(['MetroAreaID']).agg({'MedianSalesPrice': ['mean']}).reset_index()
smh.columns = ['MetroAreaID', 'Average Sale Price']
smh.head()

,MetroAreaID,Average Sale Price
0,1000,700512.282609
1,1001,541461.428571
2,1003,492651.590909
3,1004,507030.000000
4,1006,342768.846154


#### Average Resale Year Built: 
- Average year when the resale homes were built. (Example 1992) 

## Calculations

##### Age of Resales: 
- Average Resale Year Built — Current Year (e.g., 2021 -1992 = 29) 

##### Expected Resale Premium: 
- Based on the age of Resales, this is derived from lookup table #1. Example: For  age band of 21-30, Resale Premium is 40% 

##### Actual Resale Premium: 
- (Average Stanley Martin Base Price — Average Resale Price)/ Average Resale Price 

##### Score: 
- No score if there is no Actual Resale Premium. 
- If Actual Resale Premium > Expected Resale  Premium, then 1. 
- Else, if Actual Resale Premium< Expected Resale Premium then 3, else 2. 

##### Results: 
- Is Pricing Above or Below the Expected Premium to Resales? 
- To get this Premium to Resales value is looked up to Resale Premium Analysis table. (Table 2) Relative Affordability Ranking: 
- Based on the Score from the Calculations above. Lookup Table #3 and obtain the ranking.


In [ ]:
ideas:
map, show radius , show score 
inputs: radius length, input address (come with community), 